In [1]:
import os

In [2]:
%pwd

'/home/hemanth/only_ml/Kidney-Disease-Classification-Deep-Learning-Project/research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'/home/hemanth/only_ml/Kidney-Disease-Classification-Deep-Learning-Project'

In [5]:
from dataclasses import dataclass
from pathlib import Path

# whatever i've written in the config.yaml file i'am copying here
@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [19]:
%pip install boxsdk
%pip install pyyaml types-PyYAML

Note: you may need to restart the kernel to use updated packages.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 757.7/757.7 kB 824.4 kB/s eta 0:00:0036m0:00:01m eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [22]:
%pip install ensure

  Using cached ensure-1.0.4-py3-none-any.whl.metadata (10 kB)
Using cached ensure-1.0.4-py3-none-any.whl (15 kB)
Note: you may need to restart the kernel to use updated packages.


In [24]:
%pip install boxsdk

Note: you may need to restart the kernel to use updated packages.


In [27]:
%pip install python-box

  Using cached python_box-7.1.1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (7.8 kB)
Using cached python_box-7.1.1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (4.3 MB)
Note: you may need to restart the kernel to use updated packages.


In [29]:
from src.cnnClassifier.constants import *
from src.cnnClassifier.utils.common import read_yaml, create_directories

In [30]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [32]:
%pip install gdown

  Using cached gdown-5.1.0-py3-none-any.whl.metadata (5.7 kB)
Using cached gdown-5.1.0-py3-none-any.whl (17 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.9/147.9 kB 681.8 kB/s eta 0:00:0031m2.0 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [33]:
import os
import zipfile
import gdown
from src.cnnClassifier import logger
from src.cnnClassifier.utils.common import get_size

In [34]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    
    def download_file(self)-> str:
        '''
        Fetch data from the url
        '''

        try: 
            dataset_url = self.config.source_URL
            zip_download_dir = self.config.local_data_file
            os.makedirs("artifacts/data_ingestion", exist_ok=True)
            logger.info(f"Downloading data from {dataset_url} into file {zip_download_dir}")

            file_id = dataset_url.split("/")[-2]
            prefix = 'https://drive.google.com/uc?/export=download&id='
            gdown.download(prefix+file_id,zip_download_dir)

            logger.info(f"Downloaded data from {dataset_url} into file {zip_download_dir}")

        except Exception as e:
            raise e
        
    

    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)



In [35]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-03-05 09:37:38,097: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-03-05 09:37:38,124: INFO: common: yaml file: params.yaml loaded successfully]
[2024-03-05 09:37:38,125: INFO: common: created directory at: artifacts]
[2024-03-05 09:37:38,126: INFO: common: created directory at: artifacts/data_ingestion]
[2024-03-05 09:37:38,128: INFO: 3172177572: Downloading data from https://drive.google.com/file/d/1wTvlJjWavZS7kV9dlWGqYUw1-wLbZvyQ/view?usp=sharing into file artifacts/data_ingestion/data.zip]


Downloading...
From (original): https://drive.google.com/uc?/export=download&id=1wTvlJjWavZS7kV9dlWGqYUw1-wLbZvyQ
From (redirected): https://drive.google.com/uc?%2Fexport=download&id=1wTvlJjWavZS7kV9dlWGqYUw1-wLbZvyQ&confirm=t&uuid=b5e00e7b-6121-4cc9-af63-84e02b9334d3
To: /home/hemanth/only_ml/Kidney-Disease-Classification-Deep-Learning-Project/artifacts/data_ingestion/data.zip
100%|██████████| 57.7M/57.7M [00:21<00:00, 2.66MB/s]

[2024-03-05 09:38:02,309: INFO: 3172177572: Downloaded data from https://drive.google.com/file/d/1wTvlJjWavZS7kV9dlWGqYUw1-wLbZvyQ/view?usp=sharing into file artifacts/data_ingestion/data.zip]
